# 5. 워드 임베딩

## 원-핫 인코딩

In [1]:
import re
from konlpy.tag import Okt
from collections import Counter

In [2]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

- 자음의 범위 : ㄱ ~ ㅎ
- 모음의 범위 : ㅏ ~ ㅣ
- 완성형 한글의 범위 : 가 ~ 힣   
한글, 공백을 제외한 모든 문자를 표현하는 regex : `[^ㄱ-ㅎㅏ-ㅣ가-힣 ]`

In [4]:
# 전처리
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


In [5]:
# 한국어 형태소 분석기로 토크나이징
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


In [7]:
# Counter로 단어장 만들기. 빈도수로 저장된다
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [8]:
vocab['임금님']

2

In [9]:
# 빈도 수 상위 5개만 단어장으로 저장 (value=빈도수)
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [10]:
# 높은 빈도 수 일수록 낮은 정수 인덱스 부여한 단어장 (value=정수 인덱스)
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


### 원-핫 벡터 만들기

In [11]:
# 특정 단어와 단어장을 입력하면 해당 단어의 원-핫 벡터를 리턴
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index)) # 단어장의 길이만큼 영벡터 생성
       index = word2index[word] # 단어의 정수 인덱스
       one_hot_vector[index-1] = 1
       return one_hot_vector

In [12]:
one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

케라스로 원-핫 인코딩 간단히 해보자

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [14]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [15]:
# 케라스 토크나이저
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [17]:
# 단어장의 크기 저장
vocab_size = len(t.word_index) + 1
vocab_size

6

여기서 단어장 크기에 +1을 해준 이유는, 0번 단어에 패딩토큰을 넣어줄 수 있도록 비워둔 것

In [18]:
# 정수 시퀀스로 변환
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [19]:
# to_categorical()로 원-핫 벡터 시퀀스로 변환
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


vocab_size = 6 인 벡터로 변환되었다!